# Constraint Programming

_Or how to cheat at puzzles..._

#### From Wikipedia:


In computer science, constraint programming is a programming paradigm wherein relations between variables are stated in the form of constraints. Constraints differ from the common primitives of imperative programming languages in that they do not specify a step or sequence of steps to execute, but rather the properties of a solution to be found

It all starts with a problem

```
a ∈ {0,1}
b ∈ {0,1}
a > b
```

__Import antigravity...__

In [ ]:
from constraint import *

In [ ]:
problem = Problem()

problem.addVariable("a", [0,1])
problem.addVariable("b", [0,1])
problem.getSolutions()

Let's constrain the problem

In [ ]:
problem.addConstraint(lambda a, b: a > b)

In [ ]:
problem.getSolution()

## Coins

How many ways can you make $5.00

In [ ]:
problem = Problem()
total = 5.00
variables = ("0.01", "0.05", "0.10", "0.50", "1.00")
values = [float(x) for x in variables]
for variable, value in zip(variables, values):
    problem.addVariable(variable, range(int(total / value)))
problem.addConstraint(ExactSumConstraint(total, values), variables)
problem.addConstraint(ExactSumConstraint(100))
problem.getSolution()

In [ ]:
len(problem.getSolutions())

## Sudoku

__More vars, more problems__

In [ ]:
# Some value is given.
board = [[0, 9, 0, 7, 0, 0, 8, 6, 0],
         [0, 3, 1, 0, 0, 5, 0, 2, 0],
         [8, 0, 6, 0, 0, 0, 0, 0, 0],
         [0, 0, 7, 0, 5, 0, 0, 0, 6],
         [0, 0, 0, 3, 0, 7, 0, 0, 0],
         [5, 0, 0, 0, 1, 0, 7, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0, 9],
         [0, 2, 0, 6, 0, 0, 0, 5, 0],
         [0, 5, 4, 0, 0, 8, 0, 7, 0]]
board

In [ ]:
problem = Problem()

# Define the variables: 9 rows of 9 variables rangin in 1...9
for i in range(1, 10):
    problem.addVariables(range(i * 10 + 1, i * 10 + 10), range(1, 10))

# Each row has different values
for i in range(1, 10):
    problem.addConstraint(AllDifferentConstraint(), range(i * 10 + 1, i * 10 + 10))

# Each colum has different values
for i in range(1, 10):
    problem.addConstraint(AllDifferentConstraint(), range(10 + i, 100 + i, 10))

# Each 3x3 box has different values
problem.addConstraint(AllDifferentConstraint(), [11, 12, 13, 21, 22, 23, 31, 32, 33])
problem.addConstraint(AllDifferentConstraint(), [41, 42, 43, 51, 52, 53, 61, 62, 63])
problem.addConstraint(AllDifferentConstraint(), [71, 72, 73, 81, 82, 83, 91, 92, 93])

problem.addConstraint(AllDifferentConstraint(), [14, 15, 16, 24, 25, 26, 34, 35, 36])
problem.addConstraint(AllDifferentConstraint(), [44, 45, 46, 54, 55, 56, 64, 65, 66])
problem.addConstraint(AllDifferentConstraint(), [74, 75, 76, 84, 85, 86, 94, 95, 96])

problem.addConstraint(AllDifferentConstraint(), [17, 18, 19, 27, 28, 29, 37, 38, 39])
problem.addConstraint(AllDifferentConstraint(), [47, 48, 49, 57, 58, 59, 67, 68, 69])
problem.addConstraint(AllDifferentConstraint(), [77, 78, 79, 87, 88, 89, 97, 98, 99])

for i in range(1, 10):
    for j in range(1, 10):
        if initValue[i - 1][j - 1] != 0:
            problem.addConstraint(lambda var, val=board[i - 1][j - 1]:
                                  var == val, (i * 10 + j,))

# Get _a_ solution
solution = problem.getSolution()

In [ ]:
[list(solution[(10 * row) + col] for col in range(1,9)) for row in range(1,9)] 

## KenKen

It's like sudoku, but not

![kenken](https://upload.wikimedia.org/wikipedia/commons/0/0e/Kenkenproblem.png)

In [ ]:
problem = Problem()

problem.addVariable("a", range(1,7))
problem.addVariable("b", range(1,7))
problem.addConstraint(lambda a, b: a * b == 20 or b * a == 20, ("a", "b"))
problem.getSolutions()

In [ ]:
problem = Problem()

problem.addVariable("a", range(1,7))
problem.addVariable("b", range(1,7))
problem.addVariable("c", range(1,7))
problem.addVariable("d", range(1,7))

problem.addConstraint(lambda a, b, c, d: a * b * c * d == 240, ("a", "b", "c", "d"))
problem.getSolutions()

In [97]:
from itertools import permutations

kenken_board = """
11 + 11 21
2 / 12 13
20 * 14 24
6 * 15 16 26 36
3 - 22 23
3 / 25 35
240 x 31 32 41 42
6 * 33 34
6 * 43 53
7 + 44 54 55
30 * 45 46
6 * 51 52
9 + 56 66
8 + 61 62 63
2 / 64 65
"""

def constrain(value, arithmetic):
    def evaluate(*t): 
        if arithmetic == "+":
            return sum(t) == value
        elif arithmetic == "*":
            result = 1
            for x in t:
                result *= x
            return result == value
        elif arithmetic == "-":
            for p in permutations(t):
                result = p[0]
                for x in p[1:]:
                    result -= x
                if abs(result) == value:
                    return True
            return False
        elif arithmetic == "/":
            for p in permutations(t):
                result = p[0]
                for x in p[1:]:
                    result /= x
                if result == value:
                    return True
            return False
        return True
    return evaluate
            
problem = Problem()

for line in kenken_board.split('\n')[1:-1]:
    parts = line.split(' ')
    for var in parts[2:]:
        problem.addVariable(var, range(1,7))
    value = int(parts[0])
    arithmetic = parts[1]
    problem.addConstraint(constrain(value, arithmetic), parts[2:])
    
for i in range(1,7):
    problem.addConstraint(AllDifferentConstraint(), [str(i * 10 + j) for j in range(1,7)])
    problem.addConstraint(AllDifferentConstraint(), [str(j * 10 + i) for j in range(1,7)])

solution = problem.getSolution()
[list(solution[str((10 * row) + col)] for col in range(1,7)) for row in range(1,7)] 

[[5, 6, 3, 4, 1, 2],
 [6, 1, 4, 5, 2, 3],
 [4, 5, 2, 3, 6, 1],
 [3, 4, 1, 2, 5, 6],
 [2, 3, 6, 1, 4, 5],
 [1, 2, 5, 6, 3, 4]]

Spoilers

![solution](https://upload.wikimedia.org/wikipedia/commons/1/14/Kenkensolution.png)